Versionen: diese unterscheidet sich von der aktuellen in github, bei dem Dataframe rfm habe ich den Index resettet, damit ich nach CustomerIds suchen kann, um es mit Tableau zu vergliechen. Zudem habe ich das best_customers anders sortiert. 

# Recency, Frequency, Monetary

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from sklearn.cluster import KMeans

In [3]:
os.chdir(r'D:\Data\Projects\Business Analytics\E-Commerce Data')
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [4]:
from warnings import filterwarnings
filterwarnings('ignore')

In [5]:
df = pd.read_csv('dfclean.csv', parse_dates=['InvoiceDate'])
print(df.shape)
df.head()

(401604, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390,17850,United Kingdom


In [6]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID              int64
Country                object
dtype: object

In [7]:
df.CustomerID = df.CustomerID.astype('int64').astype('str')

### RFM

In [8]:
# Create Invoice column
df['Invoice'] = df.UnitPrice * df.Quantity

In [9]:
rfm = df.groupby('CustomerID').agg({'InvoiceDate': lambda date: (df.InvoiceDate.max()- date.max()).days, 
                                      'InvoiceNo': lambda num: num.nunique(),
                                      'Invoice': lambda price: price.sum()})

rfm.columns = ['recency', 'frequency', 'monetary']

In [10]:
rfm = rfm.reset_index()
rfm.head()

,CustomerID,recency,frequency,monetary
0,12346,325,2,0.000
1,12347,1,7,4310.000
2,12348,74,4,1797.240
3,12349,18,1,1757.550
4,12350,309,1,334.400


In [11]:
# Parameters
rfm.describe()

,recency,frequency,monetary
count,4372.000,4372.000,4372.000
mean,91.047,5.075,1893.531
std,100.765,9.339,8218.696
min,0.000,1.000,-4287.630
25%,16.000,1.000,291.795
50%,49.000,3.000,644.070
75%,142.000,5.000,1608.335
max,373.000,248.000,279489.020


In [12]:
df.loc[df.CustomerID == '18283'].Invoice.sum()

2045.53

In [13]:
# Quantiles
quant = rfm.quantile(q=[0.25, 0.5, 0.75])
quant

,recency,frequency,monetary
0.250,16.000,1.000,291.795
0.500,49.000,3.000,644.070
0.750,142.000,5.000,1608.335


In [14]:
quantiles = quant.to_dict()
quantiles

{'recency': {0.25: 16.0, 0.5: 49.0, 0.75: 142.0},
 'frequency': {0.25: 1.0, 0.5: 3.0, 0.75: 5.0},
 'monetary': {0.25: 291.795, 0.5: 644.0699999999999, 0.75: 1608.335}}

In [15]:
# Segmentation Table

l = []

for i in rfm.recency:
    if i <= 16:
        l.append(1)
    elif i <= 49:
        l.append(2)
    elif i <=142:
        l.append(3)
    else:
        l.append(4)
    
rfm['recency_score'] = l

In [16]:
l1 = []

for i in rfm.frequency:
    if i <= 1:
        l1.append(4)
    elif i <= 3:
        l1.append(3)
    elif i <= 5:
        l1.append(2)
    else:
        l1.append(1)
    
rfm['frequency_score'] = l1

In [17]:
l2 = []

for i in rfm.monetary:
    if i <= 291.795:
        l2.append(4)
    elif i <= 644.070:
        l2.append(3)
    elif i <= 1608.335:
        l2.append(2)
    else:
        l2.append(1)
    
rfm['monetary_score'] = l2

In [18]:
rfm.monetary_score = rfm.monetary_score.map(str)
rfm.frequency_score = rfm.frequency_score.map(str)
rfm.recency_score = rfm.recency_score.map(str)

In [19]:
rfm['RFM_Class'] = rfm.recency_score + rfm.frequency_score + rfm.monetary_score

In [20]:
rfm = rfm.sort_values(by='RFM_Class')

In [21]:
rfm.head()

,CustomerID,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_Class
2687,15984,2,11,2759.930,1,1,1,111
3869,17602,1,8,5002.220,1,1,1,111
3213,16700,8,17,5040.000,1,1,1,111
3214,16701,8,18,5150.270,1,1,1,111
1123,13854,8,29,8025.020,1,1,1,111


In [30]:
rfm.loc[rfm.CustomerID == '14646']

,CustomerID,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_Class
1703,14646,1,77,279489.020,1,1,1,111


In [23]:
best_customers = rfm.loc[rfm.RFM_Class == '111']

In [36]:
# macht warhscheinlich nur Sinn, wenn eine der Spalten gebinnt ist
best_customers.sort_values(by=['monetary', 'recency'], ascending = (False, True)).head(20)

,CustomerID,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_Class
1703,14646,1,77,279489.020,1,1,1,111
4233,18102,0,62,256438.490,1,1,1,111
3758,17450,7,55,187322.170,1,1,1,111
1895,14911,0,248,132458.730,1,1,1,111
1345,14156,9,66,113214.590,1,1,1,111
3801,17511,2,46,88125.380,1,1,1,111
3202,16684,3,31,65892.080,1,1,1,111
1005,13694,3,60,62690.540,1,1,1,111
2192,15311,0,118,59284.190,1,1,1,111
568,13089,2,118,57322.130,1,1,1,111


In [25]:
# known problem with cancellations, of the 248 226 were cancelled. 
df.loc[(df.CustomerID == '14911') & (df.InvoiceNo.str.startswith('C'))]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Invoice
22166,C539221,90185B,AMETHYST DIAMANTE EXPANDABLE RING,-3,2010-12-16 12:56:00,4.250,14911,EIRE,-12.750
22167,C539221,90185C,BLACK DIAMANTE EXPANDABLE RING,-4,2010-12-16 12:56:00,4.250,14911,EIRE,-17.000
22168,C539221,22592,CARDHOLDER HOLLY WREATH METAL,-1,2010-12-16 12:56:00,3.750,14911,EIRE,-3.750
22169,C539221,22941,CHRISTMAS LIGHTS 10 REINDEER,-12,2010-12-16 12:56:00,8.500,14911,EIRE,-102.000
22170,C539221,22942,CHRISTMAS LIGHTS 10 SANTAS,-10,2010-12-16 12:56:00,8.500,14911,EIRE,-85.000
...,...,...,...,...,...,...,...,...,...
358680,C577388,23454,THREE MINI HANGING FRAMES,-2,2011-11-18 16:58:00,4.150,14911,EIRE,-8.300
358681,C577388,21429,RED GINGHAM ROSE JEWELLERY BOX,-2,2011-11-18 16:58:00,1.950,14911,EIRE,-3.900
358682,C577388,23118,PARISIENNE JEWELLERY DRAWER,-1,2011-11-18 16:58:00,7.500,14911,EIRE,-7.500
364342,C577828,23318,BOX OF 6 MINI VINTAGE CRACKERS,-11,2011-11-22 09:53:00,2.080,14911,EIRE,-22.880
